In [1]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers , activations , models , preprocessing , utils

# read in the contents of the file
with open('/kaggle/input/faq-datasets-for-chatbot-training/faq_results.txt') as f:
    data = f.read()

# parse the JSON data
data = json.loads(data)


# Extract questions and answers from the dataset
questions = [d['Question'] for d in data]
answers = [d['Answer'] for d in data]


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
len(questions)

48

In [3]:
import pandas as pd
zippedList =  list(zip(questions, answers))
lines = pd.DataFrame(zippedList, columns = ['input' , 'output']) 
lines.head()

,input,output
0,How do I book my flight?,You can make a new booking by any of the follo...
1,What is the maximum number of seats that I can...,You can book up to nine passengers in a single...
2,How can I book a seat for my infant?,Booking an infant seat can be done through our...
3,My ticket was purchased from a travel agent. C...,"Yes, You can modify your booking under Manage ..."
4,How can I pay for my ticket online?,"You can use a credit card, debit card or Net B..."


In [4]:
input_lines = list()
for line in lines.input:
    input_lines.append( line ) 

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( input_lines ) 
tokenized_input_lines = tokenizer.texts_to_sequences( input_lines ) 

length_list = list()
for token_seq in tokenized_input_lines:
    length_list.append( len( token_seq ))
max_input_length = np.array( length_list ).max()
print( 'Input max length is {}'.format( max_input_length ))

padded_input_lines = preprocessing.sequence.pad_sequences( tokenized_input_lines , maxlen=max_input_length , padding='post' )
encoder_input_data = np.array( padded_input_lines )
print( 'Encoder input data shape -> {}'.format( encoder_input_data.shape ))

input_word_dict = tokenizer.word_index
num_input_tokens = len( input_word_dict )+1
print( 'Number of Input tokens = {}'.format( num_input_tokens))

Input max length is 33
Encoder input data shape -> (48, 33)
Number of Input tokens = 196


In [5]:
output_lines = list()
for line in lines.output:
    output_lines.append( '<START> ' + line + ' <END>' )  

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( output_lines ) 
tokenized_output_lines = tokenizer.texts_to_sequences( output_lines ) 

length_list = list()
for token_seq in tokenized_output_lines:
    length_list.append( len( token_seq ))
max_output_length = np.array( length_list ).max()
print( 'Output max length is {}'.format( max_output_length ))

padded_output_lines = preprocessing.sequence.pad_sequences( tokenized_output_lines , maxlen=max_output_length, padding='post' )
decoder_input_data = np.array( padded_output_lines )
print( 'Decoder input data shape -> {}'.format( decoder_input_data.shape ))

output_word_dict = tokenizer.word_index
num_output_tokens = len( output_word_dict )+1
print( 'Number of Output tokens = {}'.format( num_output_tokens))

Output max length is 227
Decoder input data shape -> (48, 227)
Number of Output tokens = 886


In [8]:

encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( num_input_tokens, 256 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 256 , return_state=True , recurrent_dropout=0.2 , dropout=0.2 )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( num_output_tokens, 256 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 256 , return_state=True , return_sequences=True , recurrent_dropout=0.2 , dropout=0.2)
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( num_output_tokens , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy')

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 256)    50176       ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 256)    226816      ['input_2[0][0]']                
                                                                                              

In [9]:
decoder_target_data = list()
for token_seq in tokenized_output_lines:
    decoder_target_data.append( token_seq[ 1 : ] ) 
    
padded_output_lines = preprocessing.sequence.pad_sequences( decoder_target_data , maxlen=max_output_length, padding='post' )
onehot_output_lines = utils.to_categorical( padded_output_lines , num_output_tokens )
decoder_target_data = np.array(onehot_output_lines )
print( 'Decoder target data shape -> {}'.format( decoder_target_data.shape ))

Decoder target data shape -> (48, 227, 886)


In [10]:
model.fit([encoder_input_data , decoder_input_data], decoder_target_data, batch_size=124, epochs=500) 
model.save( 'model.h5' ) 

Epoch 1/500
1/1 [==============================] - 13s 13s/step - loss: 6.7867
Epoch 2/500
1/1 [==============================] - 2s 2s/step - loss: 6.7828
Epoch 3/500
1/1 [==============================] - 1s 1s/step - loss: 6.7783
Epoch 4/500
1/1 [==============================] - 2s 2s/step - loss: 6.7726
Epoch 5/500
1/1 [==============================] - 1s 1s/step - loss: 6.7640
Epoch 6/500
1/1 [==============================] - 1s 1s/step - loss: 6.7476
Epoch 7/500
1/1 [==============================] - 1s 1s/step - loss: 6.7021
Epoch 8/500
1/1 [==============================] - 1s 1s/step - loss: 6.5841
Epoch 9/500
1/1 [==============================] - 2s 2s/step - loss: 6.4310
Epoch 10/500
1/1 [==============================] - 2s 2s/step - loss: 6.2817
Epoch 11/500
1/1 [==============================] - 2s 2s/step - loss: 6.1536
Epoch 12/500
1/1 [==============================] - 1s 1s/step - loss: 6.0554
Epoch 13/500
1/1 [==============================] - 2s 2s/step - loss: 

In [11]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=(256,))
    decoder_state_input_c = tf.keras.layers.Input(shape=(256,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [12]:
import tensorflow as tf
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( input_word_dict[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=max_input_length , padding='post')

In [21]:
enc_model , dec_model = make_inference_models()
for epoch in range( encoder_input_data.shape[0] ):
    states_values = enc_model.predict( str_to_tokens( input( 'User: ' ).replace("?","") ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = output_word_dict['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in output_word_dict.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > max_output_length:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( "Bot:" +decoded_translation.replace(' end', '') )
    print()
    


KeyboardInterrupt: Interrupted by user